In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import set_config; set_config("diagram")
import pandas as pd
import os
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

# Data récuperation (with data_test):

In [28]:
data = pd.read_csv('raw_data/data_test.csv', sep=",")
data

,Unnamed: 0,username,date,message,topic
0,0,LukeMarcheciel,29 novembre 2022 à 11:32:43,N'oubliez jamais ça les choffixENT,Grâce à l'immigration on a gagné la coupe du m...
1,1,CeintureDOrion1,29 novembre 2022 à 11:33:10,Ah bah ça va alors.,Grâce à l'immigration on a gagné la coupe du m...
2,2,Platon19,29 novembre 2022 à 11:33:26,On en aurait gagné beaucoup plus sans,Grâce à l'immigration on a gagné la coupe du m...
3,3,BanDefSemper,29 novembre 2022 à 11:34:14,On dira ça à tout ceux qui sont morts à cause...,Grâce à l'immigration on a gagné la coupe du m...
4,4,Haddock22Ready,29 novembre 2022 à 11:34:56,Et pourquoi on a l'immigration ? Car c'est un...,Grâce à l'immigration on a gagné la coupe du m...
...,...,...,...,...,...
17770,17770,BuellVaincra,29 novembre 2022 à 10:32:24,this,les TATOUAGES sont un marqueur SOCIAL
17771,17771,Jepetesapue,29 novembre 2022 à 10:53:28,Up,les TATOUAGES sont un marqueur SOCIAL
17772,17772,RoiBouna,29 novembre 2022 à 10:55:50,Et ils sont TOUS teubés et superficiels.,les TATOUAGES sont un marqueur SOCIAL
17773,17773,Jepetesapue,29 novembre 2022 à 11:03:27,Ce mec est un clown,les TATOUAGES sont un marqueur SOCIAL


# Data Preprocessing

In [29]:
import string
from nltk import word_tokenize

## Remove duplicate:

In [30]:
duplicate_count = data.duplicated().sum()
duplicate_count

0

In [31]:
# Eneleve les duplicates
data = data.drop_duplicates()
data

,Unnamed: 0,username,date,message,topic
0,0,LukeMarcheciel,29 novembre 2022 à 11:32:43,N'oubliez jamais ça les choffixENT,Grâce à l'immigration on a gagné la coupe du m...
1,1,CeintureDOrion1,29 novembre 2022 à 11:33:10,Ah bah ça va alors.,Grâce à l'immigration on a gagné la coupe du m...
2,2,Platon19,29 novembre 2022 à 11:33:26,On en aurait gagné beaucoup plus sans,Grâce à l'immigration on a gagné la coupe du m...
3,3,BanDefSemper,29 novembre 2022 à 11:34:14,On dira ça à tout ceux qui sont morts à cause...,Grâce à l'immigration on a gagné la coupe du m...
4,4,Haddock22Ready,29 novembre 2022 à 11:34:56,Et pourquoi on a l'immigration ? Car c'est un...,Grâce à l'immigration on a gagné la coupe du m...
...,...,...,...,...,...
17770,17770,BuellVaincra,29 novembre 2022 à 10:32:24,this,les TATOUAGES sont un marqueur SOCIAL
17771,17771,Jepetesapue,29 novembre 2022 à 10:53:28,Up,les TATOUAGES sont un marqueur SOCIAL
17772,17772,RoiBouna,29 novembre 2022 à 10:55:50,Et ils sont TOUS teubés et superficiels.,les TATOUAGES sont un marqueur SOCIAL
17773,17773,Jepetesapue,29 novembre 2022 à 11:03:27,Ce mec est un clown,les TATOUAGES sont un marqueur SOCIAL


## Remove unuseful columns:

In [32]:
data = data.drop(columns='Unnamed: 0')
data.sample(100)

,username,date,message,topic
16263,BowillyWonka,10 janvier 2019 à 10:41:11,Bah il y a deux versants. D'un côté des marqu...,[OFFICIEL] Le style sartorial
9290,EnfantDeFlemme,26 octobre 2022 à 11:54:54,La 2e,(Tinder) Je réveille des femmes sur la taille
9769,Deus,11 juin 2020 à 18:14:42,"On associe rien c'est juste pour le fun, ça a...",INFP est le PIRE type MBTI
16347,Sartorialisme,12 janvier 2019 à 10:36:15,Je te l'accorde le rendu peut être particulie...,[OFFICIEL] Le style sartorial
16072,Pioched,05 janvier 2019 à 12:49:52,Ça c'est moi quand j'acheterai mon premiercos...,[OFFICIEL] Le style sartorial
...,...,...,...,...
3150,LynxFrancais,14 septembre 2018 à 10:02:59,En fait faut juste faire le minimum syndical ...,Les esclaves qui branlent rien au taff.
3492,BaroudeurAlpha,29 novembre 2022 à 11:09:10,Prochaine redpill pour l'op : savoir que même...,Ça vous rend pas fou que les 3/10 femmes baise...
15552,Z-Hunter-Z,10 octobre 2022 à 16:24:10,Les Pro Poutine sur Facebook sont complètemen...,[ALERTE] Russie/Ukraine… Suite/suite
5612,JeVeuxDeceder,29 juillet 2022 à 20:02:00,C'est quoi les événements ? Les soirées ?,"[OFFICIEL] L'amphi des carabins V2 (médecine, ..."


In [7]:
missing_percent = data.isnull().sum().sort_values(ascending=False)/len(data) #NaN percentage for each column
missing_percent

username    0.035612
date        0.035443
message     0.000000
topic       0.000000
dtype: float64

## Data preprocess NLP:

In [8]:
#Preprocess data
def preprocessing(sentence):
    # enleve les espaces
    sentence = sentence.strip()
    # met en lowercase
    sentence = sentence.lower()
    # remove numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())
    # remove ponctuation
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, ' ')
    # tokenize
    sentence_tokenize = word_tokenize(sentence)
    # enleve les stop words:
    stop_words = set(stopwords.words('french'))
    tokens_cleaned = [w for w in sentence_tokenize if not w in stop_words]
    # lemmatize
    verb_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "v") for word in tokens_cleaned]
    noun_lemmatized = [WordNetLemmatizer().lemmatize(word, pos = "n") for word in verb_lemmatized]
    sentence_lemma = ' '.join(noun_lemmatized)
    return sentence_lemma

In [9]:
data['clean_message'] = data['message']
data

,username,date,message,topic,clean_message
0,LukeMarcheciel,29 novembre 2022 à 11:32:43,N'oubliez jamais ça les choffixENT,Grâce à l'immigration on a gagné la coupe du m...,N'oubliez jamais ça les choffixENT
1,CeintureDOrion1,29 novembre 2022 à 11:33:10,Ah bah ça va alors.,Grâce à l'immigration on a gagné la coupe du m...,Ah bah ça va alors.
2,Platon19,29 novembre 2022 à 11:33:26,On en aurait gagné beaucoup plus sans,Grâce à l'immigration on a gagné la coupe du m...,On en aurait gagné beaucoup plus sans
3,BanDefSemper,29 novembre 2022 à 11:34:14,On dira ça à tout ceux qui sont morts à cause...,Grâce à l'immigration on a gagné la coupe du m...,On dira ça à tout ceux qui sont morts à cause...
4,Haddock22Ready,29 novembre 2022 à 11:34:56,Et pourquoi on a l'immigration ? Car c'est un...,Grâce à l'immigration on a gagné la coupe du m...,Et pourquoi on a l'immigration ? Car c'est un...
...,...,...,...,...,...
17770,BuellVaincra,29 novembre 2022 à 10:32:24,this,les TATOUAGES sont un marqueur SOCIAL,this
17771,Jepetesapue,29 novembre 2022 à 10:53:28,Up,les TATOUAGES sont un marqueur SOCIAL,Up
17772,RoiBouna,29 novembre 2022 à 10:55:50,Et ils sont TOUS teubés et superficiels.,les TATOUAGES sont un marqueur SOCIAL,Et ils sont TOUS teubés et superficiels.
17773,Jepetesapue,29 novembre 2022 à 11:03:27,Ce mec est un clown,les TATOUAGES sont un marqueur SOCIAL,Ce mec est un clown


In [10]:
data['clean_message'] = data['clean_message'].apply(preprocessing)

In [11]:
data['clean_message']

0                             oubliez jamais ça choffixent
1                                       ah bah ça va alors
2                                 gagné beaucoup plus sans
3                            dira ça tout ceux morts cause
4        pourquoi a immigration car conséquence colonis...
                               ...                        
17770                                                 this
17771                                                   up
17772                             tous teubés superficiels
17773                                            mec clown
17774    no tatoo be the new tatoo novax notatoos futur...
Name: clean_message, Length: 17775, dtype: object

# LDA Model:

## Vectorization data:

### With Tfid:

In [14]:
vectorizer_tfid = TfidfVectorizer(max_df=20)
vectorized_data_tfid = vectorizer_tfid.fit_transform(data['clean_message'])
vectorized_data_tfid = pd.DataFrame(vectorized_data_tfid.toarray(), 
                                    columns = vectorizer_tfid.get_feature_names_out())

### With n-grams:

In [15]:
data_ngram = data['clean_message'].sample(5000)

In [17]:
vectorizer_ngram = CountVectorizer(ngram_range = (2,3))
vectorized_data_ngram = vectorizer_ngram.fit_transform(data_ngram)

vectorized_data_ngram = pd.DataFrame(
    vectorized_data_ngram.toarray(),
    columns = vectorizer_ngram.get_feature_names_out())

vectorized_data_ngram

,aaaaaahhhhhh croyais,aaaaaahhhhhh croyais allais,aaaaahhh raahh,aaaaahhh raahh rahhh,aaaah admins,aaaah admins peuvent,aaaaya animé,aaaaya animé canon,aaaaya oui,aaaaya oui crie,...,œuf donc,œuf donc clairement,œuvre meet,œuvre meet beaucoup,œuvre mettant,œuvre mettant avant,œuvre originale,œuvre originale ça,œuvre plaisirs,œuvre plaisirs richard
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Train/fit model:

### With tfid:

In [18]:
# Instantiate the LDA 
lda_model = LatentDirichletAllocation(n_components=10, max_iter = 50)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_data_tfid)

iteration: 1 of max_iter: 50
iteration: 2 of max_iter: 50
iteration: 3 of max_iter: 50
iteration: 4 of max_iter: 50
iteration: 5 of max_iter: 50
iteration: 6 of max_iter: 50
iteration: 7 of max_iter: 50
iteration: 8 of max_iter: 50
iteration: 9 of max_iter: 50
iteration: 10 of max_iter: 50
iteration: 11 of max_iter: 50
iteration: 12 of max_iter: 50
iteration: 13 of max_iter: 50
iteration: 14 of max_iter: 50
iteration: 15 of max_iter: 50
iteration: 16 of max_iter: 50
iteration: 17 of max_iter: 50
iteration: 18 of max_iter: 50
iteration: 19 of max_iter: 50
iteration: 20 of max_iter: 50
iteration: 21 of max_iter: 50
iteration: 22 of max_iter: 50
iteration: 23 of max_iter: 50
iteration: 24 of max_iter: 50
iteration: 25 of max_iter: 50
iteration: 26 of max_iter: 50
iteration: 27 of max_iter: 50
iteration: 28 of max_iter: 50
iteration: 29 of max_iter: 50
iteration: 30 of max_iter: 50
iteration: 31 of max_iter: 50
iteration: 32 of max_iter: 50
iteration: 33 of max_iter: 50
iteration: 34 of ma

LatentDirichletAllocation(max_iter=50, verbose=1)

In [19]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [20]:
print_topics(lda_model,vectorizer_tfid)

Topic 0:
[('valide', 11.061266131420313), ('weekend', 7.599539123048074), ('screenshot', 7.099999987340275), ('rager', 6.747504073663249), ('wesh', 6.646075741080492), ('fofo', 6.281901332009385), ('risitas', 6.225467949188372), ('cocca', 5.90759020362667), ('string', 5.079108782430291), ('canon', 4.9965711967003505)]
Topic 1:
[('chad', 9.013086759260542), ('lyon', 7.027556365515908), ('clos', 6.946081976897438), ('life', 6.805281847118531), ('humiliation', 5.749402930804718), ('pointstroupo', 5.5959269403966125), ('intj', 5.5494764395827465), ('pointsbrainstemglioma', 5.5099973359649805), ('pointarbrefeuillu', 5.5099973359649805), ('légende', 5.312650660748888)]
Topic 2:
[('ayaaaaaaa', 10.029661661025123), ('sieste', 9.122230229334319), ('ahiii', 8.559774371127741), ('talent', 6.993864330181184), ('kj', 5.713712454693141), ('respecte', 5.0841765740587705), ('messi', 4.968954963278186), ('evenepoel', 4.840688469615297), ('vergogne', 4.751852345901948), ('magie', 4.625178357235992)]
Top

In [21]:
def print_topics1(model, vectorizer, top_words):
    # 1. TOPIC MIXTURE OF WORDS FOR EACH TOPIC
    topic_mixture = pd.DataFrame(lda_model.components_,
                                 columns = vectorizer.get_feature_names_out())
    
    # 2. FINDING THE TOP WORDS FOR EACH TOPIC
    ## Number of topics
    n_components = topic_mixture.shape[0]
    ## Top words for each topic
    for topic in range(n_components):
        print("-"*10)
        print(f"For topic {topic}, here are the the top {top_words} words with weights:")
        topic_df = topic_mixture.iloc[topic]\
                             .sort_values(ascending = False).head(top_words)
        
        print(round(topic_df,3))

In [22]:
print_topics1(lda_model,vectorizer_tfid,5)

----------
For topic 0, here are the the top 5 words with weights:
valide        11.061
weekend        7.600
screenshot     7.100
rager          6.748
wesh           6.646
Name: 0, dtype: float64
----------
For topic 1, here are the the top 5 words with weights:
chad           9.013
lyon           7.028
clos           6.946
life           6.805
humiliation    5.749
Name: 1, dtype: float64
----------
For topic 2, here are the the top 5 words with weights:
ayaaaaaaa    10.030
sieste        9.122
ahiii         8.560
talent        6.994
kj            5.714
Name: 2, dtype: float64
----------
For topic 3, here are the the top 5 words with weights:
participant    9.156
nantes         8.993
reparti        8.904
connectés      8.769
inscription    8.322
Name: 3, dtype: float64
----------
For topic 4, here are the the top 5 words with weights:
envoie      9.492
marqueur    7.705
seum        7.463
entj        7.177
ado         7.073
Name: 4, dtype: float64
----------
For topic 5, here are the the

### With n-gram:

In [23]:
# Instantiate the LDA 
lda_model = LatentDirichletAllocation(n_components=10, max_iter = 50)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_data_ngram)

iteration: 1 of max_iter: 50
iteration: 2 of max_iter: 50
iteration: 3 of max_iter: 50
iteration: 4 of max_iter: 50
iteration: 5 of max_iter: 50
iteration: 6 of max_iter: 50
iteration: 7 of max_iter: 50
iteration: 8 of max_iter: 50
iteration: 9 of max_iter: 50
iteration: 10 of max_iter: 50
iteration: 11 of max_iter: 50
iteration: 12 of max_iter: 50
iteration: 13 of max_iter: 50
iteration: 14 of max_iter: 50
iteration: 15 of max_iter: 50
iteration: 16 of max_iter: 50
iteration: 17 of max_iter: 50
iteration: 18 of max_iter: 50
iteration: 19 of max_iter: 50
iteration: 20 of max_iter: 50
iteration: 21 of max_iter: 50
iteration: 22 of max_iter: 50
iteration: 23 of max_iter: 50
iteration: 24 of max_iter: 50
iteration: 25 of max_iter: 50
iteration: 26 of max_iter: 50
iteration: 27 of max_iter: 50
iteration: 28 of max_iter: 50
iteration: 29 of max_iter: 50
iteration: 30 of max_iter: 50
iteration: 31 of max_iter: 50
iteration: 32 of max_iter: 50
iteration: 33 of max_iter: 50
iteration: 34 of ma

LatentDirichletAllocation(max_iter=50, verbose=1)

In [24]:
print_topics(lda_model,vectorizer_ngram)

Topic 0:
[('twitter com', 19.099975478198402), ('http twitter', 15.09997147640954), ('http twitter com', 15.09997147640954), ('http www', 8.187855096160078), ('ça fait', 6.403008287145849), ('cette année', 6.10002967368727), ('peut être', 5.100024291938511), ('non plus', 5.100007456060613), ('master meef', 5.100002540102351), ('com thinktank', 5.099999999833992)]
Topic 1:
[('ça va', 18.100144845365474), ('peut être', 14.085675507750528), ('first page', 11.10001024532693), ('ça fait', 10.100068291522026), ('va faire', 8.100071256200792), ('comme ça', 7.0999318203366295), ('va être', 6.1000444967917025), ('quelque choose', 6.100041534907566), ('ça va être', 6.100027330905377), ('km alto', 6.099999999880098)]
Topic 2:
[('quelque choose', 6.100027860421155), ('aujourd hui', 5.100027777380388), ('non plus', 5.0999838464596134), ('peut être', 5.099974453937521), ('plus fort', 5.099958076719093), ('sans filtre', 4.100043386180432), ('tant mieux', 4.100031371629835), ('tout monde', 4.100021512

In [25]:
print_topics1(lda_model,vectorizer_ngram,5)

----------
For topic 0, here are the the top 5 words with weights:
twitter com         19.100
http twitter        15.100
http twitter com    15.100
http www             8.188
ça fait              6.403
Name: 0, dtype: float64
----------
For topic 1, here are the the top 5 words with weights:
ça va         18.100
peut être     14.086
first page    11.100
ça fait       10.100
va faire       8.100
Name: 1, dtype: float64
----------
For topic 2, here are the the top 5 words with weights:
quelque choose    6.1
aujourd hui       5.1
non plus          5.1
peut être         5.1
plus fort         5.1
Name: 2, dtype: float64
----------
For topic 3, here are the the top 5 words with weights:
gt gt        21.100
gt gt gt     15.100
kid buu      11.100
plus tard     8.099
non plus      7.100
Name: 3, dtype: float64
----------
For topic 4, here are the the top 5 words with weights:
team rocket     11.1
nation unies    10.1
ça fait          8.1
lt spoil gt      8.1
spoil gt         8.1
Name: 4, dtype